## This file summarizes much of the audio data
which is too long to fit NNs

In [ ]:
# common general imports
import os
import sys

# common math imports
import numpy as np
import pandas as pd

# common audio imports
import librosa
from scipy.signal import hilbert

## Custom functions to get summary statistics

In [ ]:
# differentiator
def differentiate(signal):
    return np.diff(signal, prepend=0)

# get the summary statistics
def calculate_summary_statistics(audio, sr, print_summary=False):
    """
    Calculate and print summary statistics for an audio file.
    
    Parameters
    ----------
        audio (numpy.ndarray): The audio time series.
        sr (int): The sample rate of the audio file.
        print_summary (bool): If True, prints the summary statistics.

    Returns
    -------
        dict: A dictionary containing summary statistics.

    """

    # Calculate summary statistics
    if sr == 0:
        duration = 0
    else:
        duration = len(audio) / sr
    mean_amplitude = np.mean(np.abs(audio))
    std_amplitude = np.std(audio)
    highest_peak = np.max(audio)
    lowest_valley = np.min(audio)
    energy = np.sum(audio ** 2)

    # Print summary statistics
    if print_summary:
        print(f"Duration: {duration:.2f} seconds")
        print(f"Mean Amplitude: {mean_amplitude:.4f}")
        print(f"Standard Deviation of Amplitude: {std_amplitude:.4f}")
        print(f"Highest Peak: {highest_peak:.4f}")
        print(f"Lowest Valley: {lowest_valley:.4f}")
        print(f"Energy: {energy:.4f}")

    # Save summary statistics to a dictionary
    dictionary = dict()
    dictionary['duration'] = duration
    dictionary['mean_amplitude'] = mean_amplitude
    dictionary['std_amplitude'] = std_amplitude
    dictionary['highest_peak'] = highest_peak
    dictionary['lowest_valley'] = lowest_valley
    dictionary['energy'] = energy
    return dictionary

## Custom functions to subset to the peaks

In [2]:

def max_in_windows(data, percentage=2):
    """This function computes max in non-overlapping windows of data comprising percentage of the data length.

    Parameters
    ----------
    data : array-like
        The input data to be processed.
    percentage : int, optional
        The percentage of the data length to be used for each window. Default is 2.

    Returns
    -------
    array-like
        Array of maximum values found in each window.
    """
    assert percentage > 0, "percentage must be greater than 0"
    window_size = int(len(data) * percentage / 100)
    return np.array([np.max(data[i:i + window_size]) for i in range(0, len(data), window_size)])

def argmax_in_windows(data, percentage=2):
    """This function computes the index of the maximum value in non-overlapping windows of data comprising percentage of the data length.
    Parameters
    ----------
    data : array-like
        The input data to be processed.
    percentage : int, optional
        The percentage of the data length to be used for each window. Default is 2.
    Returns
    -------
    array-like
        Array of indices of maximum values found in each window.
    """
    assert percentage > 0, "percentage must be greater than 0"
    window_size = int(len(data) * percentage / 100)
    return np.array([np.argmax(data[i:i + window_size]) for i in range(0, len(data), window_size)])

def peak_scan(data, percentage=2, num_peaks=5):
    """This function scans the data for peaks based on the maximum values in non-overlapping windows.
    Parameters
    ----------
    data : array-like
        The input data to be processed.
    percentage : int, optional
        The percentage of the data length to be used for each window. Default is 2.
    num_peaks : int, optional
        The number of peaks to be returned. Default is 5.
    Returns
    -------
    array-like
        Array of indices of the peaks found in the data.
    """
    assert num_peaks > 0, "num_peaks must be greater than 0"
    assert percentage > 0, "percentage must be greater than 0"
    max_values = max_in_windows(data, percentage)
    argmax_values = argmax_in_windows(data, percentage)
    window_size = int(len(data) * percentage / 100)
    peak_sorting = np.argsort(max_values)[::-1]
    peak_argmaxs = np.array(argmax_values)[peak_sorting]
    wind_starts = np.arange(0, len(data), window_size)
    wind_starts = wind_starts[peak_sorting]
    peak_argmaxs = peak_argmaxs + wind_starts
    return np.sort(peak_argmaxs[:num_peaks]) # do not break the temporal order of the peaks

def peak_data(data, percentage=2, num_peaks=5, peak_width=100):
    """This function extracts data around the peaks found in the data.
    Parameters
    ----------
    data : array-like
        The input data to be processed.
    percentage : int, optional
        The percentage of the data length to be used for each window. Default is 2.
    num_peaks : int, optional
        The number of peaks to be returned. Default is 5.
    peak_width : int, optional
        The width around each peak. Default is 100.
    Returns
    -------
    array-like
        Array of data segments around the peaks found in the data.
    """
    assert num_peaks > 0, "num_peaks must be greater than 0"
    assert peak_width > 0, "plateau_size must be greater than 0"
    assert percentage > 0, "percentage must be greater than 0"
    peaks = peak_scan(data, percentage, num_peaks)
    start_of_data = 0
    end_of_data = len(data)
    half_width = peak_width // 2
    peaks = np.clip(peaks, half_width, end_of_data - half_width)
    return np.array([data[p-half_width:p+half_width] for p in peaks])

def peak_zero_data(data, percentage=2, num_peaks=5, peak_width=100, rift_prop=0.2):
    """This function extracts data around the peaks and zero imputed stretches found in the data.
    Parameters
    ----------
    data : array-like
        The input data to be processed.
    percentage : int, optional
        The percentage of the data length to be used for each window. Default is 2.
    num_peaks : int, optional
        The number of peaks to be returned. Default is 5.
    peak_width : int, optional
        The width around each peak. Default is 100.
    rift_prop : float, optional
        The proportion of the peak width to be used for the zeros rift. Default is 0.2.
    Returns
    -------
    array-like
        Array of data segments around the peaks and valleys found in the data.
    """
    assert num_peaks > 0, "num_peaks must be greater than 0"
    assert peak_width > 0, "plateau_size must be greater than 0"
    assert percentage > 0, "percentage must be greater than 0"
    assert 0 < rift_prop < 1, "rift_prop must be between 0 and 1"
    peaked = peak_data(data, percentage, num_peaks, peak_width)
    rift_size = int(peak_width * rift_prop)
    rifted = np.array([np.zeros(rift_size) for _ in range(num_peaks)])
    return np.concatenate((peaked, rifted), axis=1).flatten()[:-rift_size]

## Process and summarize the audio data

In [ ]:
# Get the .ogg files for all the data
# And summarize the data
pz_data = []
sm_stat = []
fn_list = []
files_in_directories = {}
directories = os.listdir('small/train_audio')
for d in directories:
    dir_path = os.path.join('small/train_audio', d)
    files_in_directories[d] = os.listdir(dir_path)
    print(d)
    for file in files_in_directories[d]:
        assert file.endswith('.ogg')
        fn_list.append(file)
        sm_file = []
        # audio is Amplitude over time
        audio, sr = librosa.load(dir_path + '/' + file, sr=None)
        print(dir_path + '/' + file)
        # compute data around the peaks
        pz = peak_zero_data(audio, 
                            percentage=2, 
                            num_peaks=5, 
                            peak_width=100, 
                            rift_prop=0.2)
        pz_data.append(pz)
        # compute summary statistics for the peak data
        sm = list(calculate_summary_statistics(pz, 0, print_summary=False).values())
        sm_file += sm
        pz_diff = differentiate(pz)
        sm = list(calculate_summary_statistics(pz_diff, 0, print_summary=False).values())
        sm_file += sm
        # compute summary statistics for whole signal
        sm = list(calculate_summary_statistics(audio, sr, print_summary=False).values())
        sm_file += sm
        # fourier transform to frequency domain
        ft = np.fft.fft(audio)
        real_ft = ft.real
        sm = list(calculate_summary_statistics(real_ft, sr, print_summary=False).values())
        sm_file += sm
        imag_ft = ft.imag
        sm = list(calculate_summary_statistics(imag_ft, sr, print_summary=False).values())
        sm_file += sm
        # hilbert transform to get the envelope
        hilbert_transform = hilbert(audio)
        sm = list(calculate_summary_statistics(hilbert_transform, sr, print_summary=False).values())
        sm_file += sm
        # differentiate the current array signals
        differentiated_amplitude = differentiate(audio)
        sm = list(calculate_summary_statistics(differentiated_amplitude, sr, print_summary=False).values())
        sm_file += sm
        differentiated_hilbert = differentiate(hilbert_transform)
        sm = list(calculate_summary_statistics(differentiated_hilbert, sr, print_summary=False).values())
        sm_file += sm
        differentiated_real_ft = differentiate(real_ft)
        sm = list(calculate_summary_statistics(differentiated_real_ft, sr, print_summary=False).values())
        sm_file += sm
        differentiated_imag_ft = differentiate(imag_ft)
        sm = list(calculate_summary_statistics(differentiated_imag_ft, sr, print_summary=False).values())
        sm_file += sm
        sm_stat.append(sm_file)

apapan
arcter
bcnher
belkin1


## Formatting the data

In [ ]:
macro_header = ['peak_zero',
                'peak_zero_diff',
                'amplitude',
                'real_ft',
                'imag_ft',
                'hilbert',
                'amplitude_diff',
                'hilbert_diff',
                'real_ft_diff',
                'imag_ft_diff']

stats = ['duration',
         'mean',
         'std',
         'highest_peak',
         'lowest_valley',
         'energy']
header = []
for macro in macro_header:
    # for each macro, add the stats
    # to the header
    lst = [macro + '_' + stat for stat in stats]
    header += lst

In [ ]:
sm_stat_df = pd.DataFrame(sm_stat, columns=header)
sm_stat_df['file'] = fn_list
not_duration = [_ for _ in header if _.split('_')[-1] != 'duration']
sm_stat_df['duration'] = sm_stat_df['amplitude_duration']
cols = ['file', 'duration'] + not_duration
sm_stat_df = sm_stat_df[cols]
sm_stat_df.head()

,file,duration,peak_zero_mean,peak_zero_std,peak_zero_highest_peak,peak_zero_lowest_valley,peak_zero_energy,peak_zero_diff_mean,peak_zero_diff_std,peak_zero_diff_highest_peak,...,real_ft_diff_mean,real_ft_diff_std,real_ft_diff_highest_peak,real_ft_diff_lowest_valley,real_ft_diff_energy,imag_ft_diff_mean,imag_ft_diff_std,imag_ft_diff_highest_peak,imag_ft_diff_lowest_valley,imag_ft_diff_energy
0,XC174948.ogg,38.243250,0.151990,0.216105,0.992628,-0.963724,27.091248,0.113797,0.165857,1.043870,...,13.707000,26.772697,334.546875,-334.546875,8.771806e+08,13.689127,26.697587,328.448135,-314.644638,8.722657e+08
1,XC175215.ogg,14.341219,0.233658,0.285162,0.607250,-0.621231,47.173536,0.171163,0.207382,0.434898,...,7.836101,22.523485,345.603882,-345.603882,2.328130e+08,7.953264,23.254620,408.218399,-427.950836,2.481730e+08
2,XC175230.ogg,148.532250,0.546276,0.656852,1.063305,-1.056722,250.399190,0.343752,0.432396,1.221716,...,65.529773,178.978908,3540.982910,-3540.982910,1.522560e+11,65.605512,179.281997,3535.464722,-3566.416504,1.527721e+11
3,XC175233.ogg,14.341219,0.234857,0.286215,0.603929,-0.617128,47.522922,0.171730,0.208231,0.438015,...,7.834081,22.513999,345.951843,-345.951843,2.326169e+08,7.952280,23.244117,408.175674,-427.277283,2.479489e+08
4,XC175507.ogg,40.960000,0.116559,0.143177,0.305063,-0.291694,11.895361,0.080883,0.099590,0.230464,...,10.530685,29.492226,636.557098,-636.557098,1.140053e+09,10.510605,29.428464,578.873428,-618.538666,1.135129e+09


In [25]:
pz_array = np.array(pz_data)
pz_array.shape

(154, 580)

## Saving the data

In [26]:
np.save('peak_zero_data.npy', pz_array)
sm_stat_df.to_csv('summary_statistics.csv', index=False)